In [1]:
# 本文件的主要目的是看能否把地址信息转录成良好的格式，供地址解析API解析成坐标点
import pandas as pd
import numpy as np

df = pd.read_csv( 'proced_hangzhou.csv', index_col=0, parse_dates=['startTime', 'stopTime', 'duration'])

In [2]:
df.loc[:, ['scope', 'duration']]

,scope,duration
0,"浙江省-杭州市-富阳区-东洲街道(杭州富阳区东洲街道小铭建筑材料经营部,杭州富阳宏运金属压延...",0 days 01:46:00.000000000
1,浙江省-杭州市-余杭区-良渚街道-杭州艺旺包装材料有限公司、良渚博物院一带,0 days 02:41:00.000000000
2,"浙江省-杭州市-富阳区-新登镇-昌东行政村-昌东自然村,新登镇-松溪行政村(松溪家园自然村,...",0 days 00:40:36.000000000
3,"浙江省-杭州市-余杭区-乔司街道-方桥行政村(目野自然村,方桥自然村)",0 days 01:50:00.000000000
4,"浙江省-杭州市-富阳区-银湖街道(高桥行政村-舒姑垟自然村,郜村行政村-郜村自然村)",0 days 00:58:34.000000000
...,...,...
17784,"浙江省-杭州市-桐庐县-城南街道-桑园行政村(梅林路,桑园里自然村)",0 days 02:00:00.000000000
17785,浙江省-杭州市-西湖区-灵隐街道-浙大求是社区-浙大路求是村19幢、20幢、34幢、46幢、...,0 days 05:00:00.000000000
17786,浙江省-杭州市-桐庐县-横村镇-龙伏行政村-龙伏自然村,0 days 02:00:00.000000000
17787,"浙江省-杭州市-临安市-青山湖街道-庆北行政村-临北自然村,青山湖街道-孝村行政村(窑田畈自...",0 days 02:00:00.000000000


In [3]:
# 那我们的思路是，先用某种方法分割地址，因为一个停电事件可能对应到多个不同的点；这样一个停电事故的scope就变成了：(scope: [ p1, p2, .. px ]) 这么一个点列表
# 然后是将同一个scope里的点进行相近合并。 定制规则，让同一条事件中相近的点合并成为一个点

# 最后我们会得到一张格点图，每个格点有它的停电时长的值，当然还有历史值，即某次x长时间的停电距今多久了；然后对此进行建模

# 这里我们先试图绘制热力图，筛出一个区来绘制吧，比如萧山区
xsdf = df[df.scope.str.contains("萧山区")]

In [4]:
xsdf = xsdf.reset_index().drop('index', axis=1)
xsdf

,poweroffId,lineName,pubTranName,scope,startTime,typeCode,poweroffArea,poweroffReason,powerTime,cityCode,cityName,stopTime,duration
0,2019102031020050,10kV单家C167线,"罗幕村14#变,",浙江省-杭州市-萧山区-义桥镇-罗幕行政村-王家桥自然村、浙江省杭州市萧山区义桥镇王家桥农贸...,2019-10-20 09:50:16,电网故障停限电,110kV许贤变:单家C167线-罗幕村14#变台区低压开关,(低压停电信息)低压开关跳闸,2019-10-20 11:19:34,3340130,国网浙江杭州市萧山区供电有限公司,2019-10-20 12:00:16,0 days 02:10:00.000000000
1,2019102031020016,10kV新创502线,"闸北村7#变,",浙江省-杭州市-萧山区-河庄街道-闸北行政村-闸北自然村、河庄街道闸北村、闸北行政村一带,2019-10-20 07:14:18,电网故障停限电,35kV永丰变:新创502线-闸北村7#变台区低压开关,(低压停电信息)低压开关跳闸,2019-10-20 10:48:49,3340120,国网浙江杭州市钱塘新区供电公司,2019-10-20 11:40:18,0 days 04:26:00.000000000
2,2019101931020150,10kV港城A815线,"南兴村18#变,岩峰村5#变,岩峰村1#变,岩峰村3#变,岩峰村17#变,岩峰村11#变,南...","浙江省-杭州市-萧山区-南阳街道-南兴行政村(南北自然村,南兴自然村,南联自然村),南阳街道...",2019-10-19 15:40:46,电网故障停限电,110kV黎明变:港城A815线-河蓬A8152开关后段,"高压柱上断路器短路故障 原停电结束时间:2019-10-19 23:58:46,变更停电结束...",2019-10-19 19:02:56,3340130,国网浙江杭州市萧山区供电有限公司,2019-10-19 19:45:46,0 days 04:05:00.000000000
3,2019101931020109,10kV惠围A128线,"顺丰家园4#公变,",浙江省-杭州市-萧山区-宁围镇-宁围街道-丰东行政村-文明路、顺丰家园小区、顺丰家园24幢等一带,2019-10-19 14:15:23,电网故障停限电,110kV惠兴变:惠围A128线-顺丰家园4#公变台区低压开关,（低压停电信息）低压漏电保护器短路故障,2019-10-19 15:02:52,3340130,国网浙江杭州市萧山区供电有限公司,2019-10-19 15:54:23,0 days 01:39:00.000000000
4,2019101931020079,10kV国航C147线,"杭州煜辉活塞销厂,杭州公安局萧山分局,花神庙村5#变,",浙江省-杭州市-萧山区-靖江街道-花神庙社区-花神庙自然村;靖江街道-花神庙社区(杭州市公安...,2019-10-19 13:00:34,电网故障停限电,110kV梅西变:国航C147线-公安分线后段,高压柱上断路器短路故障 原停电结束时间: 2019-10-19 17:00:34，变更停电结...,2019-10-19 15:42:04,3340130,国网浙江杭州市萧山区供电有限公司,2019-10-19 16:35:34,0 days 03:35:00.000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2835,2020052631020255,10kV大同C235线,"大同三村12#变,",浙江省-杭州市-萧山区-楼塔镇-大同三行政村-大同坞自然村等,2020-05-26 15:30:38,电网故障停限电,35kV楼塔变:大同C235线-大同三村12#变低压开关,(低压停电信息)低压漏电保护器跳闸,2020-05-26 16:15:02,3340130,国网浙江杭州市萧山区供电有限公司,2020-05-26 16:45:38,0 days 01:15:00.000000000
2836,2020052631020137,10kV树蓬A530线,"安家山2#变,",浙江省-杭州市-萧山区-浦阳镇-桃源行政村-闸上自然村等,2020-05-26 11:23:04,电网故障停限电,110kV浦阳变:树蓬A530线-安家山2#变低压开关,(低压停电信息)低压漏电保护器跳闸,2020-05-26 12:22:03,3340130,国网浙江杭州市萧山区供电有限公司,2020-05-26 12:55:04,0 days 01:32:00.000000000
2837,2020061331020031,10kV北南A120线,"长龙领航城7#公变,",浙江省-杭州市-萧山区-北干街道-信息港社区-建设四路,2020-06-13 06:46:00,电网故障停限电,110kV惠兴变:北南A120线长龙领航城7#公变配变,高压开关跳闸,2020-06-13 07:48:38,3340130,国网浙江杭州市萧山区供电有限公司,2020-06-13 08:46:00,0 days 02:00:00.000000000
2838,2020061331020005,10kV山三A144线,"杭州新丝路布业有限公司,长联村16#变,1#变,杭州美兰卫浴洁具有限公司,杭州新丝路布业有限...","浙江省-杭州市-萧山区-瓜沥镇-长联行政村(长沙路,长沙自然村),瓜沥镇-解放行政村(解放自...",2020-06-13 01:56:00,电网故障停限电,220kV甘露变:山三A144线山三A1443开关后段,"高压开关跳闸 原停电结束时间为2020-06-13 07:56:00,为减少客户停电时间,加...",2020-06-13 03:43:43,3340130,国网浙江杭州市萧山区供电有限公司,2020-06-13 04:30:00,0 days 02:34:00.000000000


In [5]:
# 我们处理scope字符串的策略是，先去掉前缀，然后多点分割，最后集体用地图api解析， 写入字典，再放回去。

# 先看能不能去掉共同的 省市区前缀, 看看有无是不以这个为前缀的字符串
xsdf.scope.values

array(['浙江省-杭州市-萧山区-义桥镇-罗幕行政村-王家桥自然村、浙江省杭州市萧山区义桥镇王家桥农贸市场等一带',
       '浙江省-杭州市-萧山区-河庄街道-闸北行政村-闸北自然村、河庄街道闸北村、闸北行政村一带',
       '浙江省-杭州市-萧山区-南阳街道-南兴行政村(南北自然村,南兴自然村,南联自然村),南阳街道(岩峰行政村-岩峰自然村,横蓬行政村-横蓬自然村);南阳街道(横蓬行政村-杭州萧山亚美精细化工有限公司,南兴行政村-中铁武汉电气化局集团输变电工程有限公司)、南阳街道岩峰行政村岩峰自然村234号、浙江省杭州市萧山区南阳街道岩峰村11号变一带',
       ..., '浙江省-杭州市-萧山区-北干街道-信息港社区-建设四路',
       '浙江省-杭州市-萧山区-瓜沥镇-长联行政村(长沙路,长沙自然村),瓜沥镇-解放行政村(解放自然村,长沙路);瓜沥镇(杭州萧山腾飞布业有限公司,中国电信股份有限公司杭州萧山区分公司,杭州新丝路布业有限公司,杭州富士迪装饰材料有限公司)',
       '浙江省-杭州市-萧山区-新塘街道-傅楼行政村(傅家畈自然村,河南娄自然村,河北楼自然村)，新塘街道福楼村河北楼1号等'],
      dtype=object)

In [6]:
# 拿到不以省市区开头的字符串的索引
np.where(~ np.array([_.startswith('浙江省-杭州市-萧山区') for _ in xsdf.scope.values]))

# 看看什么情况（或者直接先扔掉？？ hhh）

# 时间紧 先扔掉吧，也不多反正

(array([ 358,  412,  464,  479,  485,  611,  612,  613,  664,  666,  667,
         678,  771,  782,  784,  791,  793,  802,  809,  898,  911,  917,
         930,  935,  957,  958,  960, 1012, 1032, 1039, 1052, 1068, 1077,
        1082, 1084, 1086, 1093, 1094, 1095, 1139, 1141, 1155, 1203, 1210,
        1213, 1225, 1227, 1282, 1307, 1311, 1334, 1336, 1339], dtype=int64),)

In [7]:
# 扔掉了
for each in np.where(~ np.array([_.startswith('浙江省-杭州市-萧山区') for _ in xsdf.scope.values]))[0]:
    xsdf = xsdf.drop( each)

In [8]:
t = xsdf['scope'].values
t = list(t)
t[:10]

['浙江省-杭州市-萧山区-义桥镇-罗幕行政村-王家桥自然村、浙江省杭州市萧山区义桥镇王家桥农贸市场等一带',
 '浙江省-杭州市-萧山区-河庄街道-闸北行政村-闸北自然村、河庄街道闸北村、闸北行政村一带',
 '浙江省-杭州市-萧山区-南阳街道-南兴行政村(南北自然村,南兴自然村,南联自然村),南阳街道(岩峰行政村-岩峰自然村,横蓬行政村-横蓬自然村);南阳街道(横蓬行政村-杭州萧山亚美精细化工有限公司,南兴行政村-中铁武汉电气化局集团输变电工程有限公司)、南阳街道岩峰行政村岩峰自然村234号、浙江省杭州市萧山区南阳街道岩峰村11号变一带',
 '浙江省-杭州市-萧山区-宁围镇-宁围街道-丰东行政村-文明路、顺丰家园小区、顺丰家园24幢等一带',
 '浙江省-杭州市-萧山区-靖江街道-花神庙社区-花神庙自然村;靖江街道-花神庙社区(杭州市公安局萧山区分局,杭州煜辉活塞销厂)一带',
 '浙江省-杭州市-萧山区-义桥镇-丁家庄行政村(丁家庄自然村,戴家里自然村);浙江省-杭州市-萧山区-义桥镇-徐童山下行政村-杭州浦阳江水泥有限公司,义桥镇-丁家庄行政村(杭州浦阳江水泥有限公司,杭州萧山交投资产经营开发有限公司)一带',
 '浙江省-杭州市-萧山区-南阳街道-横蓬行政村-横蓬自然村,南阳街道-南兴行政村(南北自然村,南兴自然村,南联自然村),南阳街道-岩峰行政村(岩峰自然村,港城大道,十米路);南阳街道-南兴行政村(华锦建设集团股份有限公司,中铁武汉电气化局集团工程有限公司),南阳街道-岩峰行政村(杭州益森实业有限公司,杭州青龙内衣厂,杭州沈士铝业有限公司,杭州锦鸿包装有限公司,杭州大宁油嘴油泵有限公司,杭州恒秀花边有限公司,杭州市萧山区人民政府南阳街道办事处),南阳街道(横蓬行政村-杭州萧山亚美精细化工有限公司,东风行政村-浙江德晨置业有限公司,龙虎行政村-杭州成邦沥青混凝土有限公司)、南阳街道岩峰行政村、南阳街道岩峰村461号一带',
 '浙江省-杭州市-萧山区-宁围镇、宁围街道(盈一行政村-盈一自然村,二桥行政村-新北自然村)等一带',
 '浙江省-杭州市-萧山区-蜀山街道-沈家里社区-沈家里路盛和苑、盛和苑118号、蜀山街道盛和苑一带',
 '浙江省-杭州市-萧山区-戴村镇-戴村行政村-戴村自然村,戴村镇-八都行政村(中潭自然村,大山坪自然村,盛家自然村,麦

In [9]:
# 去掉前缀
t = [_[12:] for _ in t]
t[:10]

['义桥镇-罗幕行政村-王家桥自然村、浙江省杭州市萧山区义桥镇王家桥农贸市场等一带',
 '河庄街道-闸北行政村-闸北自然村、河庄街道闸北村、闸北行政村一带',
 '南阳街道-南兴行政村(南北自然村,南兴自然村,南联自然村),南阳街道(岩峰行政村-岩峰自然村,横蓬行政村-横蓬自然村);南阳街道(横蓬行政村-杭州萧山亚美精细化工有限公司,南兴行政村-中铁武汉电气化局集团输变电工程有限公司)、南阳街道岩峰行政村岩峰自然村234号、浙江省杭州市萧山区南阳街道岩峰村11号变一带',
 '宁围镇-宁围街道-丰东行政村-文明路、顺丰家园小区、顺丰家园24幢等一带',
 '靖江街道-花神庙社区-花神庙自然村;靖江街道-花神庙社区(杭州市公安局萧山区分局,杭州煜辉活塞销厂)一带',
 '义桥镇-丁家庄行政村(丁家庄自然村,戴家里自然村);浙江省-杭州市-萧山区-义桥镇-徐童山下行政村-杭州浦阳江水泥有限公司,义桥镇-丁家庄行政村(杭州浦阳江水泥有限公司,杭州萧山交投资产经营开发有限公司)一带',
 '南阳街道-横蓬行政村-横蓬自然村,南阳街道-南兴行政村(南北自然村,南兴自然村,南联自然村),南阳街道-岩峰行政村(岩峰自然村,港城大道,十米路);南阳街道-南兴行政村(华锦建设集团股份有限公司,中铁武汉电气化局集团工程有限公司),南阳街道-岩峰行政村(杭州益森实业有限公司,杭州青龙内衣厂,杭州沈士铝业有限公司,杭州锦鸿包装有限公司,杭州大宁油嘴油泵有限公司,杭州恒秀花边有限公司,杭州市萧山区人民政府南阳街道办事处),南阳街道(横蓬行政村-杭州萧山亚美精细化工有限公司,东风行政村-浙江德晨置业有限公司,龙虎行政村-杭州成邦沥青混凝土有限公司)、南阳街道岩峰行政村、南阳街道岩峰村461号一带',
 '宁围镇、宁围街道(盈一行政村-盈一自然村,二桥行政村-新北自然村)等一带',
 '蜀山街道-沈家里社区-沈家里路盛和苑、盛和苑118号、蜀山街道盛和苑一带',
 '戴村镇-戴村行政村-戴村自然村,戴村镇-八都行政村(中潭自然村,大山坪自然村,盛家自然村,麦园自然村,前方自然村,马谷新村自然村),戴村镇-三头行政村(墙头自然村,大湖头自然村,凌桥自然村),戴村镇-半山行政村(丁村自然村,凌山自然村),戴村镇-马谷行政村(马谷自然村,马谷新村自然村);戴村镇(杭州市萧山区林场,杭州拂晓新材料科技

In [10]:
# 字符串十分不规则，比较棘手的是括号中的内容。
# 今天我们先省事儿，直接把括号中的内容扔掉， 只用外面的信息，然后以顿号逗号为分隔符断开

import re

In [11]:
# 写两个正则即可，然后试一试先
p_droph = re.compile( '\(.+?\)')
p_dropAnySpl = re.compile( r'，|,|、|;|；') 
p_dropg = re.compile(r'一带|等')
s = '南阳街道-南兴行政村(南北自然村,南兴自然村,南联自然村),南阳街道(岩峰行政村-岩峰自然村,横蓬行政村-横蓬自然村);南阳街道(横蓬行政村-杭州萧山亚美精细化工有限公司,南兴行政村-中铁武汉电气化局集团输变电工程有限公司)、南阳街道岩峰行政村岩峰自然村234号、浙江省杭州市萧山区南阳街道岩峰村11号变一带'
s = p_droph.sub('', s)
s

'南阳街道-南兴行政村,南阳街道;南阳街道、南阳街道岩峰行政村岩峰自然村234号、浙江省杭州市萧山区南阳街道岩峰村11号变一带'

In [12]:
s = p_dropAnySpl.sub(';', s)
s

'南阳街道-南兴行政村;南阳街道;南阳街道;南阳街道岩峰行政村岩峰自然村234号;浙江省杭州市萧山区南阳街道岩峰村11号变一带'

In [13]:
s = p_dropg.sub('', s)
s

'南阳街道-南兴行政村;南阳街道;南阳街道;南阳街道岩峰行政村岩峰自然村234号;浙江省杭州市萧山区南阳街道岩峰村11号变'

In [14]:
# ok 做一个地点全集，批量传给api；各组也要去重
addrs = set()

for i in range(len(t)):
    t[i] = p_droph.sub('', t[i])
    t[i] = p_dropAnySpl.sub(';', t[i])
    t[i] = p_dropg.sub('', t[i])
    
    
    t[i] = t[i].split(';')
    if '' in t[i]: t[i].remove('')

    t[i] = set(t[i])
    addrs = addrs.union(t[i])
    

In [15]:
f = open('addrs.txt', 'w+', encoding='utf8')
for each in addrs:
    if '杭州' not in each:
        each = '杭州'+each
    f.write(each+"\n")

In [16]:
len(t)

2787

In [17]:
len(addrs)

5339

In [18]:
addrs.remove('')

# 由于set是无序容器，而百度的这个api是没有转化前文本的；所以我们在转完后再转回来。
addrs = list(addrs)

In [19]:
import requests

url = 'http://api.map.baidu.com/geocoding/v3/?address={}&output=json&ak=7sqVWLuyZYXlxWcA4YwvZa3hFXKbfec2&callback=showLocation'
rspss = []
i = 0
for each in  addrs:
    if '杭州' not in each:
        each = '杭州市'+each
    i += 1
    print('GET:', each, i, 'of 5000')
    rsps = requests.get(url.format(each))
    rspss.append(rsps)
    

746 of 5000
GET: 杭州市宁围街道-萧山开发区国有资产经营有限公司 4747 of 5000
GET: 杭州市群欢自然村 4748 of 5000
GET: 杭州市瓜沥镇瓜沥镇殿下路119号 4749 of 5000
GET: 杭州市萧山区新街街道长山头行政村中国石油加油站 4750 of 5000
GET: 杭州市河庄街道建设行政村9组32号 4751 of 5000
GET: 杭州市临浦镇-峙山西路 4752 of 5000
GET: 杭州市盈中村 4753 of 5000
GET: 杭州市南阳街道-横蓬行政村横蓬村 4754 of 5000
GET: 靖江街道-靖港行政村(杭州永恒纺织机械有限公司 4755 of 5000
GET: 杭州市新街街道-新街镇-陈家园行政村-陈家园自然村 4756 of 5000
GET: 杭州市南庄路 4757 of 5000
GET: 杭州市城厢街道-育才东苑社区 4758 of 5000
GET: 浙江省杭州市萧山区戴村镇三头村凌桥171号 4759 of 5000
GET: (杭州涵基汽车配件有限公司 4760 of 5000
GET: 杭州政源轴承机械厂 4761 of 5000
GET: 杭州浩天物业管理有限公司 4762 of 5000
GET: 杭州市党湾镇-新梅行政村 4763 of 5000
GET: 杭州市党湾镇永安行政村2组 4764 of 5000
GET: 杭州市宁围街道-钱江社区有限公司 4765 of 5000
GET: 杭州萧山景福印染有限公司 4766 of 5000
GET: 杭州联强物资回收有限公司萧山分公司 4767 of 5000
GET: 浙江省杭州市萧山区鸿宁路401号 4768 of 5000
GET: 杭州市宁围街道-宁安社区 4769 of 5000
GET: 杭州市南闸口弄 4770 of 5000
GET: 杭州市浦阳镇-桃北新行政村 4771 of 5000
GET: 杭州市南虹路 4772 of 5000
GET: 浙江省杭州市萧山区宁围街道合丰行政村利丰路柏峰珑悦府1幢3单元802室 4773 of 5000
GET: 杭州萧山瓜沥二联校办化纤织造厂 4774 of 5000
GET: 杭州市瓜沥镇世安桥行政村世安桥自然村14号 4775 of 5000

In [20]:
# 看看结果
print(addrs[0])
rspss[0].text

浙江省杭州市萧山区新街街道双圩行政村


'showLocation&&showLocation({"status":0,"result":{"location":{"lng":120.35907126489899,"lat":30.217668800819845},"precise":0,"confidence":25,"comprehension":100,"level":"乡镇"}})'

In [21]:
import json
json.loads( re.search('\((.+)\)', rspss[0].text).group(1) )

{'status': 0,
 'result': {'location': {'lng': 120.35907126489899, 'lat': 30.217668800819844},
  'precise': 0,
  'confidence': 25,
  'comprehension': 100,
  'level': '乡镇'}}

In [22]:
# 我们先用所有地址的经纬全部替换萧山区数据，也就是 xsdf 的中的地址列scope
# 那么第一步我们要先构造 地址：经纬  大字典


''' TODO here '''

import json

addr_lng_dic = dict()

for i in range( len( addrs)):
    try:
        # print(i)
        addr_lng_dic[ addrs[i] ] = json.loads( re.search('\((.+)\)', rspss[i].text).group(1))['result']['location']
    except AttributeError:
        print('error at', i, '>>', addrs[i])
    except KeyError:
        print('no_res at', i, '>>', addrs[i])

# addr_lng_dic

# 我们可以看到，这一步也是有一些各类原因下没有相应的地点的；为了那个啥，我们在底下走替换这一步时，把这些地点筛去；直接扔掉即可



error at 248 >> 9#商场（联邦美发）
no_res at 2975 >> 浙江省-杭州市-萧山区-楼塔镇-大黄岭行政村-浙江交工集团股份有限公司浙江省杭州市萧山区楼塔镇大黄岭行政村水阁
no_res at 3419 >> 浙江省-杭州市-萧山区-经济技术开发区-市北社区-杭州萧山经济技术开发区国有资产经营有限公司
no_res at 4835 >> 浙江省-杭州市-萧山区-新街街道-山末址行政村-绍兴市柯桥区杭绍城际轨道交通建设投资有限公司
no_res at 4850 >> 浙江省-杭州市-萧山区-河庄街道-建一行政村-中国水利水电第十四工程局有限公司杭州大江东基础设施项目经理部


In [23]:
# 接下来就是把T里面的物理地址全部替换为经纬，我们用新的变量名 lng_t

# 这里注意由于有空项，不要用集合表达式为好，再细分一次操作

lng_t = list()

# for addrset in t:
#     try:
#         lng_t.append(  
#             { (addr_lng_dic[_]['lng'], addr_lng_dic[_]['lat']) for _ in addrset } 
#         )
#     except KeyError:
#         print(1)


for addrset in t:
    tmpl = set()
    for addr in addrset:
        try:
            tmpl.add(  (addr_lng_dic[addr]['lng'], addr_lng_dic[addr]['lat']) )
        except KeyError:
            pass
    lng_t.append( tmpl)

lng_t[:10]
    

[{(120.13327606648856, 30.076105166336152),
  (120.19457409889512, 30.068892958174597)},
 {(120.21551180372168, 30.25308298169347),
  (120.45334222986314, 30.31605196581798),
  (120.46848366958999, 30.338653921723363)},
 {(120.43070641860562, 30.269104550435205),
  (120.44659820124699, 30.286079095817154)},
 {(120.26905189008298, 30.220367063336294),
  (120.27011278889027, 30.221318946224738),
  (120.28698770762414, 30.21649782989135)},
 {(120.48608816061838, 30.23467467379749)},
 {(120.1837189023498, 30.07061160179425),
  (120.20862898690424, 30.06612213640747)},
 {(120.43070641860562, 30.269104550435205),
  (120.44659820124699, 30.286079095817154)},
 {(120.29232924149164, 30.24765459914221),
  (120.29830916155967, 30.202053852930828)},
 {(120.26002041631622, 30.15861032664578)},
 {(120.17029221106178, 30.005080592317373),
  (120.1989063568781, 30.021745735100378),
  (120.20626707222685, 30.01713293986793)}]

In [24]:
# 好了 替换吧
len(lng_t) - len(t) == 0
xsdf.scope = lng_t

In [25]:
# 全部走完之后，我们把位置和duration取出来，然后做一个累加，画热力图
# 我们先取它的一个视图，复制一份
pic_df = xsdf.loc[:, ('scope', 'duration')].copy()
pic_df

,scope,duration
0,"{(120.13327606648856, 30.076105166336152), (12...",0 days 02:10:00.000000000
1,"{(120.21551180372168, 30.25308298169347), (120...",0 days 04:26:00.000000000
2,"{(120.43070641860562, 30.269104550435205), (12...",0 days 04:05:00.000000000
3,"{(120.28698770762414, 30.21649782989135), (120...",0 days 01:39:00.000000000
4,"{(120.48608816061838, 30.23467467379749)}",0 days 03:35:00.000000000
...,...,...
2835,"{(120.13709483626876, 29.884171982013306)}",0 days 01:15:00.000000000
2836,"{(120.2606888133609, 29.96992060064806)}",0 days 01:32:00.000000000
2837,"{(120.25207346679166, 30.208310245168462)}",0 days 02:00:00.000000000
2838,"{(120.46626229660626, 30.196281582758512)}",0 days 02:34:00.000000000


In [26]:
# 由于这个duration是上个df里减来的（我也忘了处理过程了）；我们给它换成分钟值, 因为懒 所以我们就自己写个字符串解析了 numpy的timedelta要花时间看

def timedura_to_min( duration_string):
    """
        such as  `0 days 02:10:00.0000000` -> 0*24*60 + 2*60 +10
    """
    res = int(0)
    gs = duration_string.split(' ')
    
    res += int(gs[0]) * 24 * 60
    ts = gs[2].split(':')
    res += int(ts[0]) * 60
    res += int(ts[1])

    return res
    pass

# 先写个方法

In [27]:
# 计算下 替换回去
pic_df.duration =  [timedura_to_min( _) for _ in pic_df.duration.values]

In [28]:
# 好了！ 这就是我们想要的结果； 接下来我们再划分， 把相同的点的值相加
# 到这一步其实可以存一下当前的df；但是我们短期内是不会断开这个文件的连接的
pic_df

,scope,duration
0,"{(120.13327606648856, 30.076105166336152), (12...",130
1,"{(120.21551180372168, 30.25308298169347), (120...",266
2,"{(120.43070641860562, 30.269104550435205), (12...",245
3,"{(120.28698770762414, 30.21649782989135), (120...",99
4,"{(120.48608816061838, 30.23467467379749)}",215
...,...,...
2835,"{(120.13709483626876, 29.884171982013306)}",75
2836,"{(120.2606888133609, 29.96992060064806)}",92
2837,"{(120.25207346679166, 30.208310245168462)}",120
2838,"{(120.46626229660626, 30.196281582758512)}",154


In [58]:
# next(pic_df.iterrows())[1][x]  x项0为scope 1为duration

scope_dura_dic = dict()

for row in pic_df.iterrows():
    for s in row[1][0]:
        if s in scope_dura_dic:
            scope_dura_dic[s] += row[1][1]
        else:
            scope_dura_dic[s] = row[1][1]

print( len(scope_dura_dic) )
print( len(addr_lng_dic) )

2176
5333


In [101]:
# 从上面我们可以看到，5300多个地点其实只投射到了2200个经纬，
# 这有一部分原因是因为地址解析不准确，但也有一部分原因是一些地点本来就在一个经纬坐标上
# 解析不准确的原因一是因为我们的信息处理方法有垃圾之处，在这点是可以进行改进的

# 那么不多说，我们做高德的数据集; 高德需要四个数据项： lat lng latlng count； latlng这个项就是前俩项并成一个元组，不知道为啥要这样， count就是热力值


gd_df = pd.DataFrame(scope_dura_dic.items())
gd_df.columns = ['经纬度','count']
gd_df

,经纬度,count
0,"(120.13327606648856, 30.076105166336152)",3249
1,"(120.19457409889512, 30.068892958174597)",130
2,"(120.21551180372168, 30.25308298169347)",78562
3,"(120.46848366958999, 30.338653921723363)",37299
4,"(120.45334222986314, 30.31605196581798)",1784
...,...,...
2171,"(120.32808872204362, 30.173386241495713)",130
2172,"(120.19348032210617, 30.08119950749933)",420
2173,"(120.30455112722629, 30.17623359886891)",117
2174,"(120.25207346679166, 30.208310245168462)",120


In [106]:
# 高德不能接受元组，我们把经纬转成字符串。
gd_df['经纬度'] = [ str(_[0]) + ", " + str(_[1]) for _ in gd_df['经纬度'].values]



In [108]:
gd_df.index = gd_df['经纬度']
gd_df.loc[:, 'count'].to_csv('gddf')

In [111]:
# 至此已经走完第一个demo了；我们把gddf放入高德lab中即可；
# 为了防止反复使用api我们可以用pickle存一下rspss变量这个响应列表；当然，也不是很有必要；

# 最后是一些接下来的工作： 要保存现场；然后回顾一下以上的处理过程，
#   最好是从最原始的df开始重新温习一遍全部的处理流程。然后针对各个中段写一个比较好的代码以便复用；
#   这里立个 TODO 吧；明天来做这个工作； 
#   
#   by 7.20 night